In [11]:
import os
import glob
import pandas as pd

# Define the root directory where you want to search for sub-folders and CSV files
root_directory = "PlayerData"  # Assuming "PlayerData" is in your current working directory

# Get a list of all sub-folders
sub_folders = [f.path for f in os.scandir(root_directory) if f.is_dir()]

# Initialize a dictionary to store merged dataframes
merged_dataframes = {}

# Iterate through the sub-folders and search for CSV files
for sub_folder in sub_folders:
    # Define the pattern to search for CSV files (e.g., all files with a .csv extension)
    csv_pattern = os.path.join(sub_folder, "*.csv")
    
    # Use glob to find CSV files matching the pattern
    csv_files = glob.glob(csv_pattern)
    
    # Iterate through the CSV files and merge them by filename
    for csv_file in csv_files:
        file_name = os.path.splitext(os.path.basename(csv_file))[0]  # Extract the filename without extension
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file)
        
        # Check if a DataFrame with the same name already exists in the dictionary
        if file_name in merged_dataframes:
            # Append the data to the existing DataFrame
            merged_dataframes[file_name] = pd.concat([merged_dataframes[file_name], df], ignore_index=True)
        else:
            # Create a new entry in the dictionary for the DataFrame
            merged_dataframes[file_name] = df

# Now, the 'merged_dataframes' dictionary contains DataFrames merged by filename.
# You can access each merged DataFrame using the filename as the key.


C:\Users\Marc\AppData\Local\Temp\ipykernel_6872\552861867.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_dataframes[file_name] = pd.concat([merged_dataframes[file_name], df], ignore_index=True)
C:\Users\Marc\AppData\Local\Temp\ipykernel_6872\552861867.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_dataframes[file_name] = pd.concat([merged_dataframes[file_name], df], ignore_index=True)
C:\Users\Marc\AppData\Local\Temp\ipykernel_6872\552861867.py:32: FutureWarning: The be

In [12]:
# Assuming you have already created the 'merged_dataframes' dictionary
# Get the names of all DataFrames in 'merged_dataframes'
df_names = list(merged_dataframes.keys())

# Print the list of DataFrame names
print(df_names)


['Close', 'DAY', 'Depth Chart', 'Down', 'Field Position', 'Group', 'Half', 'Location', 'Month', 'Opponent', 'Outcome', 'Pt Diff', 'Quarter', 'Season Games', 'split', 'Surface', 'Weather', 'Victory Margin']


In [10]:
merged_dataframes['Opponent'].head()

,Opponent,REC,YDS,AVG,TD,LNG,CAR,YDS.1,AVG.1,TD.1,...,RTG,1-19,20-29,30-39,40-49,50+,FG,FG%,XP,PTS
0,vs MIA,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,vs OAK,5.0,34.0,6.8,0.0,15,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,vs WAS,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vs OAK,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vs WAS,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
